In [2]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler, RobustScaler, QuantileTransformer
from nltk.tokenize import RegexpTokenizer
from preprocess import preprocess
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

train_data = pd.read_csv(os.path.join("data", "train.csv"))
test_data = pd.read_csv(os.path.join("data", "test.csv"))
train_data["description"].fillna("", inplace=True)
train_data["host_is_superhost"].fillna("f", inplace=True)
train_data["beds"].fillna("1", inplace=True)
train_data.dropna(inplace=True)
X, y = train_data.drop(["price"], axis=1), train_data["price"]
y = y.astype(int)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=42)
X_test = test_data

X_train, X_val, X_test = preprocess(X_train), preprocess(X_val), preprocess(X_test)


In [3]:
scaler = RobustScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)

In [6]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()

hyperparameters = {
    'n_estimators': [100, 200, 300, 400, 500],
    'class_weight' : [None], 
    'max_depth': [20, 30, 40, 50],
    'min_samples_split': [5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['log2', 'sqrt'], 
    'criterion': ["gini", "entropy", "log_loss"]
}
clf = GridSearchCV(rf, hyperparameters, cv=5, verbose=3, n_jobs=20, scoring="accuracy")
best_rf = clf.fit(X_train,y_train)

Fitting 5 folds for each of 720 candidates, totalling 3600 fits
[CV 3/5] END class_weight=None, criterion=gini, max_depth=20, max_features=log2, min_samples_leaf=1, min_samples_split=5, n_estimators=100;, score=0.534 total time=   1.2s
[CV 5/5] END class_weight=None, criterion=gini, max_depth=20, max_features=log2, min_samples_leaf=1, min_samples_split=5, n_estimators=100;, score=0.533 total time=   1.6s
[CV 2/5] END class_weight=None, criterion=gini, max_depth=20, max_features=log2, min_samples_leaf=1, min_samples_split=5, n_estimators=100;, score=0.518 total time=   1.6s
[CV 1/5] END class_weight=None, criterion=gini, max_depth=20, max_features=log2, min_samples_leaf=1, min_samples_split=5, n_estimators=100;, score=0.534 total time=   1.6s
[CV 4/5] END class_weight=None, criterion=gini, max_depth=20, max_features=log2, min_samples_leaf=1, min_samples_split=5, n_estimators=100;, score=0.538 total time=   1.6s
[CV 4/5] END class_weight=None, criterion=gini, max_depth=20, max_features=l

In [7]:
from pprint import pprint
pprint(best_rf.best_estimator_.get_params())
y_pred = best_rf.predict(X_val).astype(int)
print(classification_report(y_val.astype(int), y_pred, digits=5))

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'entropy',
 'max_depth': 50,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 5,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 500,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}
              precision    recall  f1-score   support

           0    0.69048   0.87437   0.77162       796
           1    0.47004   0.42327   0.44543       593
           2    0.43548   0.26839   0.33210       503
           3    0.43758   0.60284   0.50708       564
           4    0.60687   0.41732   0.49456       381
           5    0.84184   0.66000   0.73991       250

    accuracy                        0.56560      3087
   macro avg    0.58038   0.54103   0.54845      3087
weighted avg    0.56232   0.56560   0.55225      3087



In [5]:
X_test_input = X_test.to_numpy().astype(float)
X_test_input = scaler.transform(X_test_input)
test_predictions = best_rf.predict(X_test_input)

submission = {
    "id" : list(range(len(test_predictions))), 
    "price" : list(test_predictions.astype(float))
}

submission = pd.DataFrame.from_dict(submission)

submission.to_csv(os.path.join("submissions/", "bahng_rf6.csv"), index=False)

In [50]:
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.ensemble import StackingClassifier


# param_grid = {
#     'n_estimators' : [100, 200, 300, 400, 500, 1000],
#     'learning_rate' : [0.1, 0.5, 1.0], 
#     'algorithm' : ['SAMME.R']
# }

# # Adaboost with DecisionTree as base estimator
# ada_clf = AdaBoostClassifier(
#     estimator=DecisionTreeClassifier(max_depth = 5)
# )

# gscv = GridSearchCV(ada_clf, param_grid, cv=5, verbose=3, n_jobs=20, scoring="accuracy")
# gscv.fit(X_train, y_train)
# ada_clf = gscv.best_estimator_
# ada_val_accuracy = ada_clf.score(X_val, y_val)
# print("Adaboost Validation Accuracy:", ada_val_accuracy)

# estimators = [('ridge', DecisionTreeClassifier()),
#               ('lasso', LinearSVC()),
#               ('knr', KNeighborsClassifier(n_neighbors=20,
                                        #   metric='euclidean'))]


reg = AdaBoostClassifier(
estimator=RandomForestClassifier(), n_estimators=50)

# reg = StackingClassifier(
    # estimators=estimators,
    # final_estimator=final_estimator)

reg.fit(X_train, y_train)

ada_val_accuracy = reg.score(X_val, y_val)
print("Adaboost Validation Accuracy:", ada_val_accuracy)

Adaboost Validation Accuracy: 0.5706984667802385


In [39]:
X_test_input = X_test.to_numpy().astype(float)
X_test_input = scaler.transform(X_test_input)
test_predictions = reg.predict(X_test_input)

submission = {
    "id" : list(range(len(test_predictions))), 
    "price" : list(test_predictions.astype(float).astype(int).astype(float))
}

submission = pd.DataFrame.from_dict(submission)

submission.to_csv(os.path.join("submissions/", "bahng_ada2.csv"), index=False)